# Team - Neural Networks

This notebook is used to implement a neural network on the the Principal Components from the Ethereum, Credit Card, and Insurance datasets.

In [1]:
import pandas as pd
import numpy as np

In [3]:
# read in the data for modeling
data = pd.read_pickle('Data/principal_components.pkl')
print(data.shape)

# get a single column indicating if a transaction is fraud as our target
to_scam = np.array(pd.read_pickle('Data/to_scam.pkl'))
from_scam = np.array(pd.read_pickle('Data/from_scam.pkl'))

temp = to_scam + from_scam
scam =[]
for i in temp:
    if i==0:
        scam.append(0)
    else:
        scam.append(1)
target = pd.DataFrame(scam)

(71250, 15)


## Ethereum

In [19]:
# import DNN libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras import Sequential 

In [78]:
# create DNN maker function
def create_dnn(nodes,layers,dropout,activation,input_shape):
    model = Sequential()
    
    for i in list(range(1,layers+1)):
        if i == 1:
            model.add(Dense(nodes, input_dim = (input_shape), activation = tf.math.tanh))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))
        else:
            model.add(Dense(i/2, activation='swish'))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))

    model.add(BatchNormalization())
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer = 'Adam')
    return model

In [45]:
eth_dnn_f1

[0.8714570858283434,
 0.8775469831849654,
 0.8539192399049881,
 0.8836198379766844,
 0.8617084237522193]

In [38]:
from sklearn.metrics import recall_score, f1_score, precision_score, roc_auc_score
eth_dnn_mods = []
eth_dnn_f1 = []
eth_dnn_recall = []
eth_dnn_precision = []
eth_dnn_roc = []

kfold = KFold(n_splits=5, shuffle=True)
kfold.get_n_splits(data)

for train_index, test_index in kfold.split(data):
    x_train, x_test = np.array(data)[train_index], np.array(data)[test_index]
    y_train, y_test = np.array(target)[train_index], np.array(target)[test_index]

    eth_model = create_dnn(32,2,0.3,'swish') # create a DNN with the given number of initial nodes and layers
    eth_model.fit(x_train, pd.DataFrame(y_train), epochs = 500, batch_size = 4096,verbose = 1)
    
    preds = np.round(eth_model.predict(x_test),0)
    f1 = f1_score(y_test,preds)
    recall = recall_score(y_test,preds)
    precision = precision_score(y_test,preds)
    roc = roc_auc_score(y_test,preds)
    
    eth_dnn_mods.append(eth_model)
    eth_dnn_f1.append(f1)
    eth_dnn_recall.append(recall)
    eth_dnn_precision.append(precision)
    eth_dnn_roc.append(roc)

Epoch 1/500
14/14 [==============================] - 0s 2ms/step - loss: 0.8679
Epoch 2/500
14/14 [==============================] - 0s 4ms/step - loss: 0.8559
Epoch 3/500
14/14 [==============================] - 0s 2ms/step - loss: 0.8466
Epoch 4/500
14/14 [==============================] - 0s 3ms/step - loss: 0.8410
Epoch 5/500
14/14 [==============================] - 0s 3ms/step - loss: 0.8360
Epoch 6/500
14/14 [==============================] - 0s 2ms/step - loss: 0.8316
Epoch 7/500
14/14 [==============================] - 0s 2ms/step - loss: 0.8276
Epoch 8/500
14/14 [==============================] - 0s 2ms/step - loss: 0.8234
Epoch 9/500
14/14 [==============================] - 0s 2ms/step - loss: 0.8192
Epoch 10/500
14/14 [==============================] - 0s 2ms/step - loss: 0.8154
Epoch 11/500
14/14 [==============================] - 0s 2ms/step - loss: 0.8113
Epoch 12/500
14/14 [==============================] - 0s 2ms/step - loss: 0.8072
Epoch 13/500
14/14 [=================

## Insurance

In [67]:
data = pd.read_csv('insurance_claims_preprocessed.csv')
data.pop('Unnamed: 0')
target = data.pop('target')
data.head()

,months_as_customer,policy_deductable,umbrella_limit,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,...,collision_type_Side Collision,incident_severity_Minor Damage,incident_severity_Total Loss,incident_severity_Trivial Damage,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Other,authorities_contacted_Police,property_damage_YES,police_report_available_YES
0,1.078140,-0.222383,-0.479476,1.011331,0.953851,-0.956261,-0.823865,0.009759,0.461838,-0.189283,...,1,0,0,0,0,0,0,1,1,1
1,0.208995,1.412784,1.697980,-0.901927,0.953851,-0.524475,-0.823865,-1.210174,-1.338700,-1.363822,...,0,1,0,0,0,0,0,1,0,0
2,-0.608002,1.412784,1.697980,0.358023,0.953851,-0.668404,1.140056,1.229693,1.362107,0.054644,...,0,1,0,0,0,0,0,1,0,0
3,0.452355,1.412784,2.133471,0.853388,-1.267577,-0.956261,-0.823865,0.009759,0.461838,-0.224130,...,0,0,0,0,0,0,0,1,0,0
4,0.208995,-0.222383,2.133471,1.467210,-0.683741,1.202666,-0.823865,-1.210174,-0.438431,-1.257232,...,0,1,0,0,0,1,0,0,0,0


In [68]:
target_class =[]
for i in target:
    if i == 'Y':
        target_class.append(1)
    else:
        target_class.append(0)
target_class = np.array(target_class)

array([1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,

In [70]:
from sklearn.metrics import recall_score, f1_score, precision_score, roc_auc_score
insur_dnn_mods = []
insur_dnn_f1 = []
insur_dnn_recall = []
insur_dnn_precision = []
insur_dnn_roc = []

kfold = KFold(n_splits=5, shuffle=True)
kfold.get_n_splits(data)

for train_index, test_index in kfold.split(data):
    x_train, x_test = np.array(data)[train_index], np.array(data)[test_index]
    y_train, y_test = np.array(target_class)[train_index], np.array(target_class)[test_index]

    insur_model = create_dnn(32,2,0.3,'swish') # create a DNN with the given number of initial nodes and layers
    insur_model.fit(x_train, pd.DataFrame(y_train), epochs = 500, batch_size = 4096,verbose = 1)
    
    preds = np.round(insur_model.predict(x_test),0)
    f1 = f1_score(y_test,preds)
    recall = recall_score(y_test,preds)
    precision = precision_score(y_test,preds)
    roc = roc_auc_score(y_test,preds)
    
    insur_dnn_mods.append(eth_model)
    insur_dnn_f1.append(f1)
    insur_dnn_recall.append(recall)
    insur_dnn_precision.append(precision)
    insur_dnn_roc.append(roc)

Epoch 1/500
1/1 [==============================] - 0s 798us/step - loss: 0.8508
Epoch 2/500
1/1 [==============================] - 0s 656us/step - loss: 0.8525
Epoch 3/500
1/1 [==============================] - 0s 706us/step - loss: 0.8479
Epoch 4/500
1/1 [==============================] - 0s 673us/step - loss: 0.8494
Epoch 5/500
1/1 [==============================] - 0s 787us/step - loss: 0.8506
Epoch 6/500
1/1 [==============================] - 0s 1ms/step - loss: 0.8519
Epoch 7/500
1/1 [==============================] - 0s 763us/step - loss: 0.8461
Epoch 8/500
1/1 [==============================] - 0s 842us/step - loss: 0.8483
Epoch 9/500
1/1 [==============================] - 0s 767us/step - loss: 0.8473
Epoch 10/500
1/1 [==============================] - 0s 697us/step - loss: 0.8477
Epoch 11/500
1/1 [==============================] - 0s 767us/step - loss: 0.8501
Epoch 12/500
1/1 [==============================] - 0s 686us/step - loss: 0.8475
Epoch 13/500
1/1 [=====================

## Credit Card

In [75]:
# read in the data for modeling and remove unneeded columns
data_credit = pd.read_csv('creditcard.csv')
print(data_credit.shape)
data_credit.pop('Time')
target_class = data_credit.pop('Class')
data_credit.head()

(284807, 31)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [76]:
# standardize value column
import sklearn
from sklearn.preprocessing import StandardScaler

transformer = StandardScaler().fit(np.array(data_credit['Amount']).reshape(-1,1))
transformed_value = transformer.transform(np.array(data_credit['Amount']).reshape(-1,1)).reshape(1,284807)[0]
data_credit['Amount']=transformed_value
data_credit.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403


In [79]:
from sklearn.metrics import recall_score, f1_score, precision_score, roc_auc_score
credit_dnn_mods = []
credit_dnn_f1 = []
credit_dnn_recall = []
credit_dnn_precision = []
credit_dnn_roc = []

kfold = KFold(n_splits=5, shuffle=True)
kfold.get_n_splits(data_credit)

for train_index, test_index in kfold.split(data_credit):
    x_train, x_test = np.array(data_credit)[train_index], np.array(data_credit)[test_index]
    y_train, y_test = np.array(target_class)[train_index], np.array(target_class)[test_index]

    credit_model = create_dnn(32,2,0.3,'swish',data_credit.shape[1]) # create a DNN with the given number of initial nodes and layers
    credit_model.fit(x_train, pd.DataFrame(y_train), epochs = 500, batch_size = 4096,verbose = 1)
    
    preds = np.round(credit_model.predict(x_test),0)
    f1 = f1_score(y_test,preds)
    recall = recall_score(y_test,preds)
    precision = precision_score(y_test,preds)
    roc = roc_auc_score(y_test,preds)
    
    credit_dnn_mods.append(credit_model)
    credit_dnn_f1.append(f1)
    credit_dnn_recall.append(recall)
    credit_dnn_precision.append(precision)
    credit_dnn_roc.append(roc)

Epoch 1/500
56/56 [==============================] - 0s 3ms/step - loss: 0.9514
Epoch 2/500
56/56 [==============================] - 0s 3ms/step - loss: 0.9269
Epoch 3/500
56/56 [==============================] - 0s 3ms/step - loss: 0.9009
Epoch 4/500
56/56 [==============================] - 0s 3ms/step - loss: 0.8774
Epoch 5/500
56/56 [==============================] - 0s 3ms/step - loss: 0.8567
Epoch 6/500
56/56 [==============================] - 0s 3ms/step - loss: 0.8386
Epoch 7/500
56/56 [==============================] - 0s 3ms/step - loss: 0.8227
Epoch 8/500
56/56 [==============================] - 0s 3ms/step - loss: 0.8090
Epoch 9/500
56/56 [==============================] - 0s 3ms/step - loss: 0.7975
Epoch 10/500
56/56 [==============================] - 0s 3ms/step - loss: 0.7872
Epoch 11/500
56/56 [==============================] - 0s 3ms/step - loss: 0.7777
Epoch 12/500
56/56 [==============================] - 0s 3ms/step - loss: 0.7684
Epoch 13/500
56/56 [=================

## Evaluation

In [87]:
# save average metrics across all splits and repeats
eth_dnn_f1_mean = np.mean(eth_dnn_f1)
eth_dnn_recall_mean = np.mean(eth_dnn_recall)
eth_dnn_precision_mean = np.mean(eth_dnn_precision)
eth_dnn_roc_mean = np.mean(eth_dnn_roc)

eth_dnn_f1_var = np.var(eth_dnn_f1)
eth_dnn_recall_var = np.var(eth_dnn_recall)
eth_dnn_precision_var = np.var(eth_dnn_precision)
eth_dnn_roc_var = np.var(eth_dnn_roc)

eth_dnn_f1_max = np.max(eth_dnn_f1)
eth_dnn_recall_max = np.max(eth_dnn_recall)
eth_dnn_precision_max = np.max(eth_dnn_precision)
eth_dnn_roc_max = np.max(eth_dnn_roc)

eth_dnn_f1_min = np.min(eth_dnn_f1)
eth_dnn_recall_min = np.min(eth_dnn_recall)
eth_dnn_precision_min = np.min(eth_dnn_precision)
eth_dnn_roc_min = np.min(eth_dnn_roc)

eth_dnn_results = pd.DataFrame()
eth_dnn_results['F1'] = [eth_dnn_f1_mean,eth_dnn_f1_var,eth_dnn_f1_max,eth_dnn_f1_min]
eth_dnn_results['Recall'] = [eth_dnn_recall_mean,eth_dnn_recall_var,eth_dnn_recall_max,eth_dnn_recall_min]
eth_dnn_results['Precision'] = [eth_dnn_precision_mean,eth_dnn_precision_var,eth_dnn_precision_max,eth_dnn_precision_min]
eth_dnn_results['Roc'] = [eth_dnn_roc_mean,eth_dnn_roc_var,eth_dnn_roc_max,eth_dnn_roc_min]
eth_dnn_results.rename(index={0:'Mean',1:'Variance',2:'Max',3:'Min'})

,F1,Recall,Precision,Roc
Mean,0.869650,0.770542,0.998275,0.885104
Variance,0.000114,0.000274,0.000004,0.000069
Max,0.883620,0.792346,1.000000,0.896042
Min,0.853919,0.745078,0.994536,0.872539


In [88]:
# save average metrics across all splits and repeats
credit_dnn_f1_mean = np.mean(credit_dnn_f1)
credit_dnn_recall_mean = np.mean(credit_dnn_recall)
credit_dnn_precision_mean = np.mean(credit_dnn_precision)
credit_dnn_roc_mean = np.mean(credit_dnn_roc)

credit_dnn_f1_var = np.var(credit_dnn_f1)
credit_dnn_recall_var = np.var(credit_dnn_recall)
credit_dnn_precision_var = np.var(credit_dnn_precision)
credit_dnn_roc_var = np.var(credit_dnn_roc)

credit_dnn_f1_max = np.max(credit_dnn_f1)
credit_dnn_recall_max = np.max(credit_dnn_recall)
credit_dnn_precision_max = np.max(credit_dnn_precision)
credit_dnn_roc_max = np.max(credit_dnn_roc)

credit_dnn_f1_min = np.min(credit_dnn_f1)
credit_dnn_recall_min = np.min(credit_dnn_recall)
credit_dnn_precision_min = np.min(credit_dnn_precision)
credit_dnn_roc_min = np.min(credit_dnn_roc)

credit_dnn_results = pd.DataFrame()
credit_dnn_results['F1'] = [credit_dnn_f1_mean,credit_dnn_f1_var,credit_dnn_f1_max,credit_dnn_f1_min]
credit_dnn_results['Recall'] = [credit_dnn_recall_mean,credit_dnn_recall_var,credit_dnn_recall_max,credit_dnn_recall_min]
credit_dnn_results['Precision'] = [credit_dnn_precision_mean,credit_dnn_precision_var,credit_dnn_precision_max,credit_dnn_precision_min]
credit_dnn_results['Roc'] = [credit_dnn_roc_mean,credit_dnn_roc_var,credit_dnn_roc_max,credit_dnn_roc_min]
credit_dnn_results.rename(index={0:'Mean',1:'Variance',2:'Max',3:'Min'})

,F1,Recall,Precision,Roc
Mean,0.812674,0.783481,0.850596,0.891623
Variance,0.001385,0.005780,0.001266,0.001444
Max,0.847222,0.884058,0.910256,0.941906
Min,0.744898,0.675926,0.813333,0.837831


In [90]:
# save average metrics across all splits and repeats
insur_dnn_f1_mean = np.mean(insur_dnn_f1)
insur_dnn_recall_mean = np.mean(insur_dnn_recall)
insur_dnn_precision_mean = np.mean(insur_dnn_precision)
insur_dnn_roc_mean = np.mean(insur_dnn_roc)

insur_dnn_f1_var = np.var(insur_dnn_f1)
insur_dnn_recall_var = np.var(insur_dnn_recall)
insur_dnn_precision_var = np.var(insur_dnn_precision)
insur_dnn_roc_var = np.var(insur_dnn_roc)

insur_dnn_f1_max = np.max(insur_dnn_f1)
insur_dnn_recall_max = np.max(insur_dnn_recall)
insur_dnn_precision_max = np.max(insur_dnn_precision)
insur_dnn_roc_max = np.max(insur_dnn_roc)

insur_dnn_f1_min = np.min(insur_dnn_f1)
insur_dnn_recall_min = np.min(insur_dnn_recall)
insur_dnn_precision_min = np.min(insur_dnn_precision)
insur_dnn_roc_min = np.min(insur_dnn_roc)

insur_dnn_results = pd.DataFrame()
insur_dnn_results['F1'] = [insur_dnn_f1_mean,insur_dnn_f1_var,insur_dnn_f1_max,insur_dnn_f1_min]
insur_dnn_results['Recall'] = [insur_dnn_recall_mean,insur_dnn_recall_var,insur_dnn_recall_max,insur_dnn_recall_min]
insur_dnn_results['Precision'] = [insur_dnn_precision_mean,insur_dnn_precision_var,insur_dnn_precision_max,insur_dnn_precision_min]
insur_dnn_results['Roc'] = [insur_dnn_roc_mean,insur_dnn_roc_var,insur_dnn_roc_max,insur_dnn_roc_min]
insur_dnn_results.rename(index={0:'Mean',1:'Variance',2:'Max',3:'Min'})

,F1,Recall,Precision,Roc
Mean,0.483123,0.390399,0.639818,0.658612
Variance,0.000364,0.000705,0.002514,0.000171
Max,0.507042,0.428571,0.720000,0.672925
Min,0.450000,0.346154,0.562500,0.639293
